## 📦 BƯỚC 0: Cài Đặt và Import Thư Viện

In [ ]:
# Cài đặt thư viện 
import subprocess
import sys

def install_if_needed(package):
    try:
        __import__(package.replace('-', '_'))
        print(f'✅ {package} đã cài đặt')
    except ImportError:
        print(f'📥 Đang cài đặt {package}...')
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', package, '-q'])
        print(f'✅ {package} cài đặt xong')

# Cài đặt các package 
packages = ['biopython', 'lightgbm', 'scikit-learn', 'pandas', 'numpy', 'tqdm']
for pkg in packages:
    install_if_needed(pkg)

print('\n✅ Tất cả thư viện sẵn sàng!')

📥 Đang cài đặt biopython...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 41.6 MB/s eta 0:00:00
✅ biopython cài đặt xong
✅ lightgbm đã cài đặt
📥 Đang cài đặt scikit-learn...
✅ scikit-learn cài đặt xong
✅ pandas đã cài đặt
✅ numpy đã cài đặt
✅ tqdm đã cài đặt

✅ Tất cả thư viện sẵn sàng!


In [2]:
# Import thư viện
import pandas as pd
import numpy as np
import os
import pickle
import time
import warnings
from pathlib import Path
from collections import Counter
from tqdm import tqdm

# BioPython
from Bio import SeqIO

# Sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import hamming_loss

# PyTorch
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

warnings.filterwarnings('ignore')

# Check GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print('✅ Tất cả thư viện đã được import!')
print(f'📊 NumPy: {np.__version__} | Pandas: {pd.__version__}')
print(f'🔥 PyTorch: {torch.__version__} | Device: {device}')

✅ Tất cả thư viện đã được import!
📊 NumPy: 1.26.4 | Pandas: 2.2.3
🔥 PyTorch: 2.6.0+cu124 | Device: cpu


## 📁 BƯỚC 1: Thiết Lập Đường Dẫn

In [ ]:
# Phát hiện môi trường
if os.path.exists('/kaggle/input'):
    print('🔍 Đang chạy trên Kaggle')
    
    # List all available datasets
    input_base = Path('/kaggle/input')
    print(f'   Available inputs: {os.listdir(input_base)}')
    
    # Find CAFA dataset
    cafa_dir = None
    for item in os.listdir(input_base):
        if 'cafa' in item.lower():
            cafa_dir = input_base / item
            break
    
    if cafa_dir is None:
        cafa_dir = input_base / os.listdir(input_base)[0]
    
    print(f'   Dataset dir: {cafa_dir}')
    print(f'   Contents: {os.listdir(cafa_dir)}')
    
    # Check structure 
    if (cafa_dir / 'Train').exists():
        TRAIN_PATH = cafa_dir / 'Train'
        TEST_PATH = cafa_dir / 'Test'
    else:
        # Files directly in cafa_dir
        TRAIN_PATH = cafa_dir
        TEST_PATH = cafa_dir
    
    IA_PATH = cafa_dir / 'IA.tsv'
    OUTPUT_PATH = Path('/kaggle/working')
    DATA_PATH = OUTPUT_PATH / 'data'
    MODEL_PATH = OUTPUT_PATH / 'models'
    SUBMISSION_PATH = OUTPUT_PATH  
    
else:
    print('🔍 Đang chạy Local')
    TRAIN_PATH = Path('../Train')
    TEST_PATH = Path('../Test')
    IA_PATH = Path('../IA.tsv')
    
    DATA_PATH = Path('../data/processed')
    MODEL_PATH = Path('../models')
    SUBMISSION_PATH = Path('../submissions')

# Tạo thư mục output
for path in [DATA_PATH, MODEL_PATH, SUBMISSION_PATH]:
    path.mkdir(parents=True, exist_ok=True)

print(f'\n📁 Train: {TRAIN_PATH}')
print(f'📁 Test: {TEST_PATH}')

# Verify key files exist
print(f'\n📋 File verification:')
for fname in ['train_terms.tsv', 'train_sequences.fasta']:
    fpath = TRAIN_PATH / fname
    exists = fpath.exists()
    print(f'   {fname}: {"✅" if exists else "❌"} {fpath}')
    
    if not exists:
        # Search for file
        print(f'   🔍 Searching for {fname}...')
        for root, dirs, files in os.walk('/kaggle/input'):
            if fname in files:
                print(f'      Found: {root}/{fname}')
                if 'train_terms' in fname:
                    TRAIN_PATH = Path(root)

print('\n✅ Đường dẫn đã thiết lập!')

🔍 Đang chạy trên Kaggle
   Available inputs: ['cafa-6']
   Dataset dir: /kaggle/input/cafa-6
   Contents: ['sample_submission.tsv', 'IA.tsv', 'Test', 'Train']

📁 Train: /kaggle/input/cafa-6/Train
📁 Test: /kaggle/input/cafa-6/Test

📋 File verification:
   train_terms.tsv: ✅ /kaggle/input/cafa-6/Train/train_terms.tsv
   train_sequences.fasta: ✅ /kaggle/input/cafa-6/Train/train_sequences.fasta

✅ Đường dẫn đã thiết lập!


## 📥 BƯỚC 2: Tải Dữ Liệu

In [ ]:
print('='*60)
print('📂 LOADING DATA')
print('='*60)

def extract_protein_id(fasta_id):
    """
    Extract UniProt accession từ FASTA ID
    Handles multiple formats:
    - "sp|A0A0C5B5G6|MOTSC_HUMAN" -> "A0A0C5B5G6"
    - "A0A0C5B5G6 9606" -> "A0A0C5B5G6"
    - "A0A0C5B5G6" -> "A0A0C5B5G6"
    """
    fasta_id = fasta_id.split()[0]
    
    if '|' in fasta_id:
        parts = fasta_id.split('|')
        return parts[1] if len(parts) >= 2 else fasta_id
    
    return fasta_id

# 1. Load annotations (train_terms.tsv)
print('📋 Loading train_terms.tsv...')
train_terms = pd.read_csv(TRAIN_PATH / 'train_terms.tsv', sep='\t')
print(f'   ✅ {len(train_terms):,} annotations')
print(f'   Columns: {list(train_terms.columns)}')

# Standardize column names
column_mapping = {}
if 'EntryID' in train_terms.columns:
    column_mapping['EntryID'] = 'protein_id'
if 'term' in train_terms.columns:
    column_mapping['term'] = 'go_term'

if column_mapping:
    train_terms = train_terms.rename(columns=column_mapping)
    print(f'   📝 Renamed columns: {column_mapping}')
    print(f'   New columns: {list(train_terms.columns)}')

# 2. Load protein sequences
print('\n🧬 Loading train_sequences.fasta...')
train_sequences = {}
for record in SeqIO.parse(TRAIN_PATH / 'train_sequences.fasta', 'fasta'):
    accession = extract_protein_id(record.id)
    train_sequences[accession] = str(record.seq)
print(f'   ✅ {len(train_sequences):,} sequences')

# Debug: Show first few IDs
print(f'   Sample FASTA IDs: {list(train_sequences.keys())[:3]}')
print(f'   Sample train_terms IDs: {train_terms["protein_id"].head(3).tolist()}')

# 3. Load test sequences
print('\n🧪 Loading testsuperset.fasta...')
test_sequences = {}
for record in SeqIO.parse(TEST_PATH / 'testsuperset.fasta', 'fasta'):
    accession = extract_protein_id(record.id)
    test_sequences[accession] = str(record.seq)
print(f'   ✅ {len(test_sequences):,} test sequences')

# 4. Load IA weights 
print('\n⚖️ Loading IA.tsv...')
try:
    ia_weights = pd.read_csv(IA_PATH, sep='\t', header=None, names=['go_term', 'ia_weight'])
    print(f'   ✅ {len(ia_weights):,} GO term weights')
except:
    ia_weights = None
    print('   ⚠️ IA.tsv not found')

# Verify overlap
train_ids = set(train_terms['protein_id'].unique())
seq_ids = set(train_sequences.keys())
overlap = train_ids & seq_ids
print(f'\n🔍 ID Matching Check:')
print(f'   train_terms IDs: {len(train_ids):,}')
print(f'   train_sequences IDs: {len(seq_ids):,}')
print(f'   Overlapping IDs: {len(overlap):,}')

if len(overlap) == 0:
    print('   ⚠️ WARNING: No matching IDs! Check ID format.')
    print(f'\n   Debug - train_terms samples: {list(train_ids)[:3]}')
    print(f'   Debug - sequence samples: {list(seq_ids)[:3]}')
elif len(overlap) < len(train_ids):
    print(f'   ⚠️ Missing sequences for {len(train_ids) - len(overlap):,} proteins')
else:
    print('   ✅ All proteins have sequences!')

print('\n' + '='*60)
print('✅ DATA LOADED SUCCESSFULLY!')
print('='*60)

📂 LOADING DATA
📋 Loading train_terms.tsv...
   ✅ 537,027 annotations
   Columns: ['EntryID', 'term', 'aspect']
   📝 Renamed columns: {'EntryID': 'protein_id', 'term': 'go_term'}
   New columns: ['protein_id', 'go_term', 'aspect']

🧬 Loading train_sequences.fasta...
   ✅ 82,404 sequences
   Sample FASTA IDs: ['A0A0C5B5G6', 'A0JNW5', 'A0JP26']
   Sample train_terms IDs: ['Q5W0B1', 'Q5W0B1', 'Q5W0B1']

🧪 Loading testsuperset.fasta...
   ✅ 224,309 test sequences

⚖️ Loading IA.tsv...
   ✅ 40,122 GO term weights

🔍 ID Matching Check:
   train_terms IDs: 82,404
   train_sequences IDs: 82,404
   Overlapping IDs: 82,404
   ✅ All proteins have sequences!

✅ DATA LOADED SUCCESSFULLY!


In [ ]:
# LOAD GO HIERARCHY (go-basic.obo)
print('='*60)
print('🌳 LOADING GO HIERARCHY')
print('='*60)

def parse_obo_file(obo_path):
    """Parse GO OBO file to extract hierarchy"""
    go_parents = {}  
    go_names = {}   
    go_namespace = {}  
    obsolete_terms = set()
    
    current_term = None
    current_name = None
    current_namespace = None
    current_parents = []
    is_obsolete = False
    
    with open(obo_path, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            
            if line == '[Term]':
                # Save previous term
                if current_term and not is_obsolete:
                    go_parents[current_term] = current_parents
                    go_names[current_term] = current_name
                    go_namespace[current_term] = current_namespace
                elif current_term and is_obsolete:
                    obsolete_terms.add(current_term)
                
                # Reset
                current_term = None
                current_name = None
                current_namespace = None
                current_parents = []
                is_obsolete = False
                
            elif line.startswith('id: GO:'):
                current_term = line.split('id: ')[1].strip()
                
            elif line.startswith('name: '):
                current_name = line.split('name: ')[1].strip()
                
            elif line.startswith('namespace: '):
                current_namespace = line.split('namespace: ')[1].strip()
                
            elif line.startswith('is_a: GO:'):
                parent = line.split('is_a: ')[1].split(' !')[0].strip()
                current_parents.append(parent)
                
            elif line.startswith('relationship: part_of GO:'):
                parent = line.split('part_of ')[1].split(' !')[0].strip()
                current_parents.append(parent)
                
            elif line == 'is_obsolete: true':
                is_obsolete = True
        
        # Save last term
        if current_term and not is_obsolete:
            go_parents[current_term] = current_parents
            go_names[current_term] = current_name
            go_namespace[current_term] = current_namespace
    
    return go_parents, go_names, go_namespace, obsolete_terms

def get_all_ancestors(term, go_parents, cache=None):
    """Get all ancestors of a GO term (including itself)"""
    if cache is None:
        cache = {}
    
    if term in cache:
        return cache[term]
    
    ancestors = {term}
    if term in go_parents:
        for parent in go_parents[term]:
            ancestors.update(get_all_ancestors(parent, go_parents, cache))
    
    cache[term] = ancestors
    return ancestors

def build_ancestor_cache(go_terms, go_parents):
    """Build ancestor cache for all GO terms"""
    cache = {}
    for term in tqdm(go_terms, desc='Building GO hierarchy'):
        get_all_ancestors(term, go_parents, cache)
    return cache

# Load GO hierarchy
obo_path = TRAIN_PATH / 'go-basic.obo'
if obo_path.exists():
    print(f'📂 Loading: {obo_path}')
    go_parents, go_names, go_namespace, obsolete_terms = parse_obo_file(obo_path)
    
    print(f'   ✅ GO terms: {len(go_parents):,}')
    print(f'   ✅ Obsolete terms: {len(obsolete_terms):,}')
    
    # Count by namespace
    ns_counts = {}
    for term, ns in go_namespace.items():
        ns_counts[ns] = ns_counts.get(ns, 0) + 1
    print(f'   📊 By namespace:')
    for ns, count in ns_counts.items():
        print(f'      • {ns}: {count:,}')
    
    # Build ancestor cache
    print('\n🔗 Building ancestor cache...')
    all_training_terms = set(train_terms['go_term'].unique())
    ancestor_cache = build_ancestor_cache(all_training_terms, go_parents)
    
    # Count average ancestors
    avg_ancestors = np.mean([len(ancestor_cache.get(t, {t})) for t in all_training_terms])
    print(f'   ✅ Average ancestors per term: {avg_ancestors:.1f}')
    
    GO_HIERARCHY_LOADED = True
else:
    print(f'⚠️ go-basic.obo not found at {obo_path}')
    GO_HIERARCHY_LOADED = False
    go_parents = {}
    ancestor_cache = {}

print('\n' + '='*60)

🌳 LOADING GO HIERARCHY
📂 Loading: /kaggle/input/cafa-6/Train/go-basic.obo
   ✅ GO terms: 40,122
   ✅ Obsolete terms: 7,979
   📊 By namespace:
      • biological_process: 25,950
      • molecular_function: 10,131
      • cellular_component: 4,040
      • external: 1

🔗 Building ancestor cache...


Building GO hierarchy: 100%|██████████| 26125/26125 [00:00<00:00, 64010.69it/s]

   ✅ Average ancestors per term: 13.3



## 📊 BƯỚC 3: Phân Tích Dữ Liệu

In [6]:
print('='*60)
print('📊 DATA ANALYSIS')
print('='*60)

# Thống kê cơ bản
n_proteins = train_terms['protein_id'].nunique()
n_go_terms = train_terms['go_term'].nunique()
n_annotations = len(train_terms)

print(f'\n🧬 Proteins: {n_proteins:,}')
print(f'📚 Unique GO Terms: {n_go_terms:,}')
print(f'📋 Total Annotations: {n_annotations:,}')
print(f'📈 Avg GO terms per protein: {n_annotations / n_proteins:.2f}')

# Phân bố GO terms
go_counts = train_terms['go_term'].value_counts()
print(f'\n📊 GO Term Distribution:')
print(f'   • Rare (1-10 proteins): {(go_counts <= 10).sum():,} terms')
print(f'   • Medium (11-100): {((go_counts > 10) & (go_counts <= 100)).sum():,} terms')
print(f'   • Common (>100): {(go_counts > 100).sum():,} terms')
print(f'   • Most common: {go_counts.index[0]} ({go_counts.iloc[0]:,} proteins)')

# Sequence lengths
if train_sequences:
    seq_lengths = [len(seq) for seq in train_sequences.values()]
    print(f'\n🧵 Sequence Length Stats:')
    print(f'   • Mean: {np.mean(seq_lengths):.0f}')
    print(f'   • Median: {np.median(seq_lengths):.0f}')
    print(f'   • Min/Max: {min(seq_lengths)} / {max(seq_lengths)}')

📊 DATA ANALYSIS

🧬 Proteins: 82,404
📚 Unique GO Terms: 26,125
📋 Total Annotations: 537,027
📈 Avg GO terms per protein: 6.52

📊 GO Term Distribution:
   • Rare (1-10 proteins): 18,868 terms
   • Medium (11-100): 6,594 terms
   • Common (>100): 663 terms
   • Most common: GO:0005515 (33,713 proteins)

🧵 Sequence Length Stats:
   • Mean: 526
   • Median: 409
   • Min/Max: 3 / 35213


## ⚙️ BƯỚC 4: Feature Engineering

Tạo các đặc trưng từ chuỗi protein:
1. **Amino Acid Composition (AAC)**: Tỷ lệ 20 amino acids
2. **Dipeptide Composition (DPC)**: Tỷ lệ 400 dipeptides
3. **Physicochemical Properties**: Các tính chất vật lý/hóa học
4. **Sequence Statistics**: Chiều dài, entropy, etc.

In [ ]:
print('='*60)
print('⚙️ FEATURE ENGINEERING (ENHANCED)')
print('='*60)

# Amino acids
AMINO_ACIDS = 'ACDEFGHIKLMNPQRSTVWY'
AA_TO_IDX = {aa: i for i, aa in enumerate(AMINO_ACIDS)}

# Physicochemical properties
HYDROPHOBIC = set('AILMFVWP')
HYDROPHILIC = set('RKDENQ')
POLAR = set('STNQ')
CHARGED_POS = set('RKH')
CHARGED_NEG = set('DE')
AROMATIC = set('FYW')
SMALL = set('AGSTC')
ALIPHATIC = set('ILV')
SULFUR = set('CM')
HYDROXYL = set('ST')

# Molecular weights
AA_MW = {'A': 89, 'C': 121, 'D': 133, 'E': 147, 'F': 165, 'G': 75, 'H': 155, 
         'I': 131, 'K': 146, 'L': 131, 'M': 149, 'N': 132, 'P': 115, 'Q': 146,
         'R': 174, 'S': 105, 'T': 119, 'V': 117, 'W': 204, 'Y': 181}

# Hydrophobicity scale (Kyte-Doolittle)
HYDRO_SCALE = {'A': 1.8, 'C': 2.5, 'D': -3.5, 'E': -3.5, 'F': 2.8, 'G': -0.4,
               'H': -3.2, 'I': 4.5, 'K': -3.9, 'L': 3.8, 'M': 1.9, 'N': -3.5,
               'P': -1.6, 'Q': -3.5, 'R': -4.5, 'S': -0.8, 'T': -0.7, 'V': 4.2,
               'W': -0.9, 'Y': -1.3}

def extract_features_enhanced(sequence):
    """Extract comprehensive features from protein sequence"""
    seq = ''.join(aa for aa in sequence.upper() if aa in AMINO_ACIDS)
    length = len(seq)
    
    if length == 0:
        return np.zeros(150) 
    
    features = []
    
    # 1. Sequence length features (3)
    features.append(length / 10000)
    features.append(np.log1p(length) / 10)
    features.append(min(length, 1000) / 1000)  # Capped length
    
    # 2. Amino Acid Composition (20)
    aa_counts = Counter(seq)
    for aa in AMINO_ACIDS:
        features.append(aa_counts.get(aa, 0) / length)
    
    # 3. Physicochemical properties (15)
    features.append(sum(1 for aa in seq if aa in HYDROPHOBIC) / length)
    features.append(sum(1 for aa in seq if aa in HYDROPHILIC) / length)
    features.append(sum(1 for aa in seq if aa in POLAR) / length)
    features.append(sum(1 for aa in seq if aa in CHARGED_POS) / length)
    features.append(sum(1 for aa in seq if aa in CHARGED_NEG) / length)
    features.append(sum(1 for aa in seq if aa in AROMATIC) / length)
    features.append(sum(1 for aa in seq if aa in SMALL) / length)
    features.append(sum(1 for aa in seq if aa in ALIPHATIC) / length)
    features.append(sum(1 for aa in seq if aa in SULFUR) / length)
    features.append(sum(1 for aa in seq if aa in HYDROXYL) / length)
    
    # Net charge
    pos_charge = sum(1 for aa in seq if aa in CHARGED_POS)
    neg_charge = sum(1 for aa in seq if aa in CHARGED_NEG)
    features.append((pos_charge - neg_charge) / length)
    
    # Molecular weight
    mw = sum(AA_MW.get(aa, 110) for aa in seq)
    features.append(mw / (length * 200))  
    
    # Hydrophobicity stats
    hydro_values = [HYDRO_SCALE.get(aa, 0) for aa in seq]
    features.append(np.mean(hydro_values) / 5)
    features.append(np.std(hydro_values) / 5)
    features.append(np.max(hydro_values) / 5 if hydro_values else 0)
    
    # 4. Sequence complexity (3)
    aa_freq = np.array([aa_counts.get(aa, 0) / length for aa in AMINO_ACIDS])
    aa_freq = aa_freq[aa_freq > 0]
    entropy = -np.sum(aa_freq * np.log2(aa_freq)) if len(aa_freq) > 0 else 0
    features.append(entropy / 4.5)
    
    # Unique AA ratio
    features.append(len(aa_counts) / 20)
    
    # Repeat ratio
    repeats = sum(1 for i in range(len(seq)-1) if seq[i] == seq[i+1])
    features.append(repeats / max(1, length - 1))
    
    # 5. N-terminal and C-terminal signals (16)
    for term_seq, name in [(seq[:50], 'N'), (seq[-50:], 'C')]:
        term_len = len(term_seq)
        if term_len > 0:
            features.append(sum(1 for aa in term_seq if aa in HYDROPHOBIC) / term_len)
            features.append(sum(1 for aa in term_seq if aa in CHARGED_POS) / term_len)
            features.append(sum(1 for aa in term_seq if aa in CHARGED_NEG) / term_len)
            features.append(sum(1 for aa in term_seq if aa in POLAR) / term_len)
            features.append(sum(1 for aa in term_seq if aa in AROMATIC) / term_len)
            features.append(sum(1 for aa in term_seq if aa in SMALL) / term_len)
            # Hydrophobicity
            term_hydro = [HYDRO_SCALE.get(aa, 0) for aa in term_seq]
            features.append(np.mean(term_hydro) / 5)
            features.append(np.std(term_hydro) / 5 if len(term_hydro) > 1 else 0)
        else:
            features.extend([0] * 8)
    
    # 6. Dipeptide features (30 most informative)
    important_dipeptides = [
        'LL', 'AA', 'VV', 'GG', 'SS', 'EE', 'KK', 'AL', 'LA', 'EK',
        'KE', 'DE', 'ED', 'RK', 'LE', 'EL', 'AE', 'EA', 'LK', 'KL',
        'GS', 'SG', 'PP', 'PG', 'GP', 'VL', 'LV', 'IL', 'LI', 'TT'
    ]
    for dp in important_dipeptides:
        count = seq.count(dp)
        features.append(count / max(1, length - 1))
    
    # 7. Tripeptide features (20 key patterns)
    important_tripeptides = [
        'LLL', 'AAA', 'GGG', 'SSS', 'EEE', 'KKK', 'PPP', 'VVV',
        'LEK', 'EKL', 'ALA', 'GAS', 'VAL', 'ILE', 'PRO', 'GLY',
        'SER', 'THR', 'ASP', 'GLU'
    ]
    for tp in important_tripeptides:
        count = seq.count(tp)
        features.append(count / max(1, length - 2))
    
    # 8. Position-specific features (20)
    # Split sequence into 5 regions
    region_size = length // 5 if length >= 5 else length
    for i in range(5):
        start = i * region_size
        end = start + region_size if i < 4 else length
        region = seq[start:end]
        if len(region) > 0:
            features.append(sum(1 for aa in region if aa in HYDROPHOBIC) / len(region))
            features.append(sum(1 for aa in region if aa in CHARGED_POS) / len(region))
            features.append(sum(1 for aa in region if aa in CHARGED_NEG) / len(region))
            features.append(sum(1 for aa in region if aa in AROMATIC) / len(region))
        else:
            features.extend([0] * 4)
    
    # 9. Pseudo Amino Acid Composition (23)
    # Sequence-order correlation factors
    for lag in [1, 2, 3, 5, 10]:
        if length > lag:
            # Hydrophobicity correlation
            h_corr = sum(HYDRO_SCALE.get(seq[i], 0) * HYDRO_SCALE.get(seq[i+lag], 0) 
                        for i in range(length - lag)) / (length - lag)
            features.append(h_corr / 25)
        else:
            features.append(0)
    
    # Type I PseAAC features
    for d in [1, 2, 3]:  
        if length > d:
            hydro_diff = sum(abs(HYDRO_SCALE.get(seq[i], 0) - HYDRO_SCALE.get(seq[i+d], 0)) 
                            for i in range(length - d)) / (length - d)
            mw_diff = sum(abs(AA_MW.get(seq[i], 110) - AA_MW.get(seq[i+d], 110)) 
                         for i in range(length - d)) / (length - d)
            features.append(hydro_diff / 10)
            features.append(mw_diff / 200)
            
            # Charge transition
            charge_trans = sum(1 for i in range(length - d) 
                              if (seq[i] in CHARGED_POS) != (seq[i+d] in CHARGED_POS)) / (length - d)
            features.append(charge_trans)
            
            # Polarity transition
            polar_trans = sum(1 for i in range(length - d)
                             if (seq[i] in POLAR) != (seq[i+d] in POLAR)) / (length - d)
            features.append(polar_trans)
            
            # Size transition
            size_trans = sum(1 for i in range(length - d)
                            if (seq[i] in SMALL) != (seq[i+d] in SMALL)) / (length - d)
            features.append(size_trans)
            
            # Aromatic clustering
            arom_cluster = sum(1 for i in range(length - d)
                              if seq[i] in AROMATIC and seq[i+d] in AROMATIC) / (length - d)
            features.append(arom_cluster)
        else:
            features.extend([0] * 6)
    
    return np.array(features, dtype=np.float32)

# Test feature extraction
test_seq = list(train_sequences.values())[0]
test_features = extract_features_enhanced(test_seq)
print(f'\n🔧 Feature vector size: {len(test_features)}')
print(f'   Sample features (first 10): {test_features[:10]}')
print(f'   Non-zero features: {np.sum(test_features != 0)}')

⚙️ FEATURE ENGINEERING (ENHANCED)

🔧 Feature vector size: 150
   Sample features (first 10): [0.0016     0.28332132 0.016      0.         0.         0.
 0.0625     0.0625     0.0625     0.        ]
   Non-zero features: 83


In [ ]:
# Extract features for all training proteins
print('\n🧬 Extracting features for training proteins...')

proteins = train_terms['protein_id'].unique()
all_go_terms = train_terms['go_term'].unique()

print(f'   Proteins in train_terms: {len(proteins):,}')
print(f'   Sequences available: {len(train_sequences):,}')

# Debug: Check matching
matched = sum(1 for p in proteins if p in train_sequences)
print(f'   Proteins with sequences: {matched:,}')

if matched == 0:
    print('\n❌ ERROR: No proteins match! Checking IDs...')
    print(f'   First 3 train_terms IDs: {proteins[:3].tolist()}')
    print(f'   First 3 sequence IDs: {list(train_sequences.keys())[:3]}')
    raise ValueError("No matching protein IDs between train_terms and train_sequences!")

# Extract features with ENHANCED function
X_features = []
valid_proteins = []

for pid in tqdm(proteins, desc='Extracting features'):
    seq = train_sequences.get(pid, '')
    if seq:
        X_features.append(extract_features_enhanced(seq))
        valid_proteins.append(pid)

X = np.array(X_features)
print(f'\n✅ Features extracted: {X.shape}')
print(f'   Valid proteins: {len(valid_proteins):,} / {len(proteins):,}')

if len(valid_proteins) == 0:
    raise ValueError("No features extracted! Check protein ID matching.")

# 📊 Tạo label matrix - VECTORIZED
print('\n📊 Creating label matrix (vectorized)...')

protein_to_idx = {p: i for i, p in enumerate(valid_proteins)}
term_to_idx = {t: i for i, t in enumerate(all_go_terms)}

# Vectorized approach
valid_mask = train_terms['protein_id'].isin(protein_to_idx)
valid_terms_df = train_terms[valid_mask]

row_indices = valid_terms_df['protein_id'].map(protein_to_idx).values
col_indices = valid_terms_df['go_term'].map(term_to_idx).values

y = np.zeros((len(valid_proteins), len(all_go_terms)), dtype=np.int8)
y[row_indices, col_indices] = 1

print(f'✅ Label matrix: {y.shape}')
print(f'   Positive labels: {y.sum():,} ({100*y.sum()/(y.shape[0]*y.shape[1]):.4f}%)')


🧬 Extracting features for training proteins...
   Proteins in train_terms: 82,404
   Sequences available: 82,404
   Proteins with sequences: 82,404


Extracting features: 100%|██████████| 82404/82404 [04:22<00:00, 314.51it/s]



✅ Features extracted: (82404, 150)
   Valid proteins: 82,404 / 82,404

📊 Creating label matrix (vectorized)...
✅ Label matrix: (82404, 26125)
   Positive labels: 537,027 (0.0249%)


In [ ]:
#  Scale Features & Filter GO Terms & Create IA Weights
print('\n📐 Scaling features...')
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
print(f'✅ Features scaled: {X_scaled.shape}')

# Filter GO terms
MIN_PROTEINS_PER_TERM = 20  

term_counts = y.sum(axis=0)
valid_mask = term_counts >= MIN_PROTEINS_PER_TERM

print(f'\n📊 GO Term Filtering:')
print(f'   • Total: {len(term_counts):,}')
print(f'   • With ≥{MIN_PROTEINS_PER_TERM} proteins: {valid_mask.sum():,}')

y_filtered = y[:, valid_mask]
filtered_go_terms = all_go_terms[valid_mask]

print(f'✅ Filtered: {y_filtered.shape[1]:,} GO terms')

#  Prepare IA Weights for Loss Function
print('\n⚖️ Preparing IA weights for weighted loss...')

if ia_weights is not None:
    ia_dict = dict(zip(ia_weights['go_term'], ia_weights['ia_weight']))
    term_ia_weights = np.array([ia_dict.get(t, 1.0) for t in filtered_go_terms])
    # Normalize weights
    term_ia_weights = term_ia_weights / term_ia_weights.mean()
    print(f'   IA weights range: [{term_ia_weights.min():.3f}, {term_ia_weights.max():.3f}]')
else:
    term_ia_weights = np.ones(len(filtered_go_terms))
    print('   Using uniform weights (IA.tsv not found)')

#  Calculate Class Weights for Imbalanced Data
print('\n⚖️ Calculating class weights...')
pos_counts = y_filtered.sum(axis=0)
neg_counts = y_filtered.shape[0] - pos_counts

# Effective number of samples weighting
beta = 0.9999
pos_weights = (1 - beta) / (1 - np.power(beta, pos_counts + 1))
neg_weights = (1 - beta) / (1 - np.power(beta, neg_counts + 1))

# Combine with IA weights
class_weights = (neg_weights / pos_weights) * term_ia_weights
class_weights = np.clip(class_weights, 0.5, 50)
print(f'   Class weights range: [{class_weights.min():.2f}, {class_weights.max():.2f}]')


📐 Scaling features...
✅ Features scaled: (82404, 150)

📊 GO Term Filtering:
   • Total: 26,125
   • With ≥20 proteins: 4,227
✅ Filtered: 4,227 GO terms

⚖️ Preparing IA weights for weighted loss...
   IA weights range: [0.000, 7.435]

⚖️ Calculating class weights...
   Class weights range: [0.50, 0.67]


## 🤖 BƯỚC 5: Training Neural Network

Sử dụng Neural Network với Residual Blocks cho multi-label classification.

In [ ]:
print('='*60)
print('🤖 MODEL TRAINING SETUP (IMPROVED FOR HIGHER SCORE)')
print('='*60)

import gc

# CẤU HÌNH - Optimized for better performance
MAX_GO_TERMS = 2000      
HIDDEN_DIM = 768         
BATCH_SIZE = 128        
NUM_EPOCHS = 40          
LEARNING_RATE = 3e-4     
WEIGHT_DECAY = 0.01      

print(f'\n⚙️ Configuration (Optimized for Score):')
print(f'   • Max GO terms: {MAX_GO_TERMS}')
print(f'   • Hidden dim: {HIDDEN_DIM}')
print(f'   • Epochs: {NUM_EPOCHS}')
print(f'   • Batch size: {BATCH_SIZE}')

# Clear unused memory
gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()

# Train-validation split
print('\n📊 Splitting data...')
X_train, X_val, y_train, y_val = train_test_split(
    X_scaled, y_filtered,
    test_size=0.1,
    random_state=42
)
print(f'   Train: {X_train.shape[0]:,} samples')
print(f'   Val: {X_val.shape[0]:,} samples')

# Select top GO terms
term_counts = y_train.sum(axis=0)
top_term_indices = np.argsort(term_counts)[-MAX_GO_TERMS:]
y_train = y_train[:, top_term_indices]
y_val = y_val[:, top_term_indices]
filtered_go_terms_final = filtered_go_terms[top_term_indices]
final_class_weights = class_weights[top_term_indices]

# Free memory from filtered arrays
del y_filtered
gc.collect()

print(f'   GO terms: {y_train.shape[1]:,}')
print(f'   Positive labels in train: {y_train.sum():,}')

🤖 MODEL TRAINING SETUP (IMPROVED FOR HIGHER SCORE)

⚙️ Configuration (Optimized for Score):
   • Max GO terms: 2000
   • Hidden dim: 768
   • Epochs: 40
   • Batch size: 128

📊 Splitting data...
   Train: 74,163 samples
   Val: 8,241 samples
   GO terms: 2,000
   Positive labels in train: 328,678


In [ ]:
#  Neural Network Model (DEEPER & STRONGER)
class SEBlock(nn.Module):
    """Squeeze-and-Excitation block for channel attention"""
    def __init__(self, dim, reduction=16):
        super().__init__()
        self.fc = nn.Sequential(
            nn.Linear(dim, dim // reduction),
            nn.ReLU(),
            nn.Linear(dim // reduction, dim),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        scale = self.fc(x)
        return x * scale

class ResidualBlock(nn.Module):
    def __init__(self, dim, dropout=0.2):
        super().__init__()
        self.block = nn.Sequential(
            nn.Linear(dim, dim),
            nn.LayerNorm(dim),  # LayerNorm thay BatchNorm 
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(dim, dim),
            nn.LayerNorm(dim)
        )
        self.se = SEBlock(dim)
        self.act = nn.GELU()
        
    def forward(self, x):
        residual = x
        out = self.block(x)
        out = self.se(out)
        return self.act(out + residual)

class ProteinClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_classes):
        super().__init__()
        
        # Input projection
        self.input_proj = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.LayerNorm(hidden_dim),
            nn.GELU(),
            nn.Dropout(0.2),
            nn.Linear(hidden_dim, hidden_dim),
            nn.LayerNorm(hidden_dim),
            nn.GELU(),
            nn.Dropout(0.2),
        )
        
        # More Residual blocks
        self.res_blocks = nn.Sequential(
            ResidualBlock(hidden_dim, 0.15),
            ResidualBlock(hidden_dim, 0.15),
            ResidualBlock(hidden_dim, 0.15),
            ResidualBlock(hidden_dim, 0.15),
            ResidualBlock(hidden_dim, 0.10),
            ResidualBlock(hidden_dim, 0.10),
        )
        
        # Output layers với bottleneck
        self.output = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.LayerNorm(hidden_dim // 2),
            nn.GELU(),
            nn.Dropout(0.1),
            nn.Linear(hidden_dim // 2, hidden_dim // 4),
            nn.LayerNorm(hidden_dim // 4),
            nn.GELU(),
            nn.Linear(hidden_dim // 4, num_classes)
        )
        
    def forward(self, x):
        x = self.input_proj(x)
        x = self.res_blocks(x)
        return self.output(x)

#  Asymmetric Loss - Better for Multi-Label
class AsymmetricLoss(nn.Module):
    """Asymmetric Loss for Multi-Label Classification"""
    def __init__(self, gamma_neg=4, gamma_pos=1, clip=0.05, pos_weight=None):
        super().__init__()
        self.gamma_neg = gamma_neg
        self.gamma_pos = gamma_pos
        self.clip = clip
        self.pos_weight = pos_weight

    def forward(self, inputs, targets):
        # Sigmoid
        xs_pos = torch.sigmoid(inputs)
        xs_neg = 1 - xs_pos

        # Asymmetric Clipping
        if self.clip is not None and self.clip > 0:
            xs_neg = (xs_neg + self.clip).clamp(max=1)

        # Basic BCE
        los_pos = targets * torch.log(xs_pos.clamp(min=1e-8))
        los_neg = (1 - targets) * torch.log(xs_neg.clamp(min=1e-8))
        
        # Asymmetric focusing
        pt = xs_pos * targets + xs_neg * (1 - targets)
        gamma = self.gamma_pos * targets + self.gamma_neg * (1 - targets)
        asymmetric_w = (1 - pt).pow(gamma)
        
        loss = -asymmetric_w * (los_pos + los_neg)
        
        # Apply pos_weight
        if self.pos_weight is not None:
            loss = loss * (targets * self.pos_weight + (1 - targets))
        
        return loss.mean()

#  Prepare DataLoaders
train_dataset = TensorDataset(
    torch.FloatTensor(X_train), 
    torch.FloatTensor(y_train)
)
val_dataset = TensorDataset(
    torch.FloatTensor(X_val), 
    torch.FloatTensor(y_val)
)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE * 2, shuffle=False, num_workers=0, pin_memory=True)

#  Training with Label Smoothing & Better Scheduling
print('\n🏋️ Training Neural Network (Deeper Model)...')
print(f'   Using device: {device}')

model = ProteinClassifier(X_train.shape[1], HIDDEN_DIM, y_train.shape[1]).to(device)
print(f'   Model parameters: {sum(p.numel() for p in model.parameters()):,}')

# Optimizer với weight decay riêng cho bias
no_decay = ['bias', 'LayerNorm.weight', 'LayerNorm.bias']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
     'weight_decay': WEIGHT_DECAY},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
     'weight_decay': 0.0}
]
optimizer = optim.AdamW(optimizer_grouped_parameters, lr=LEARNING_RATE)

# OneCycleLR - often better than cosine
total_steps = len(train_loader) * NUM_EPOCHS
scheduler = optim.lr_scheduler.OneCycleLR(
    optimizer, 
    max_lr=LEARNING_RATE * 3,
    total_steps=total_steps,
    pct_start=0.1,
    anneal_strategy='cos'
)

# Asymmetric Loss - better for imbalanced multi-label
pos_weight_tensor = torch.FloatTensor(final_class_weights).to(device)
criterion = AsymmetricLoss(gamma_neg=4, gamma_pos=1, clip=0.05, pos_weight=pos_weight_tensor)

start_time = time.time()
best_val_loss = float('inf')
best_val_f1 = 0
patience = 10
patience_counter = 0

for epoch in range(NUM_EPOCHS):
    # Train
    model.train()
    train_loss = 0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        
        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        
        optimizer.step()
        scheduler.step()
        train_loss += loss.item()
    
    # Validate
    model.eval()
    val_loss = 0
    all_val_probs = []
    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            outputs = model(X_batch)
            val_loss += criterion(outputs, y_batch).item()
            all_val_probs.append(torch.sigmoid(outputs).cpu().numpy())
    
    # Calculate F1 for early stopping
    val_probs_epoch = np.vstack(all_val_probs)
    val_preds_epoch = (val_probs_epoch >= 0.05).astype(np.int8)
    if val_preds_epoch.sum() > 0:
        from sklearn.metrics import f1_score
        val_f1 = f1_score(y_val, val_preds_epoch, average='micro', zero_division=0)
    else:
        val_f1 = 0
    
    avg_train = train_loss / len(train_loader)
    avg_val = val_loss / len(val_loader)
    current_lr = optimizer.param_groups[0]['lr']
    
    # Early stopping based on F1
    if val_f1 > best_val_f1:
        best_val_f1 = val_f1
        best_val_loss = avg_val
        best_model_state = model.state_dict().copy()
        patience_counter = 0
    else:
        patience_counter += 1
    
    if (epoch + 1) % 5 == 0 or epoch == NUM_EPOCHS - 1:
        print(f'   Epoch {epoch+1:2d}/{NUM_EPOCHS} | Train: {avg_train:.4f} | Val: {avg_val:.4f} | F1: {val_f1:.4f} | LR: {current_lr:.6f}')
    
    if patience_counter >= patience:
        print(f'\n   ⚠️ Early stopping at epoch {epoch+1}')
        break

# Load best model
model.load_state_dict(best_model_state)
training_time = time.time() - start_time

print(f'\n✅ Training completed in {training_time:.1f}s ({training_time/60:.1f} min)')
print(f'   Best Val F1: {best_val_f1:.4f}')


🏋️ Training Neural Network (Deeper Model)...
   Using device: cpu
   Model parameters: 9,018,800
   Epoch  5/40 | Train: 0.0011 | Val: 0.0011 | F1: 0.0044 | LR: 0.000898
   Epoch 10/40 | Train: 0.0010 | Val: 0.0010 | F1: 0.0044 | LR: 0.000840
   Epoch 15/40 | Train: 0.0009 | Val: 0.0010 | F1: 0.0044 | LR: 0.000708
   Epoch 20/40 | Train: 0.0008 | Val: 0.0010 | F1: 0.0046 | LR: 0.000528
   Epoch 25/40 | Train: 0.0008 | Val: 0.0010 | F1: 0.0050 | LR: 0.000333
   Epoch 30/40 | Train: 0.0007 | Val: 0.0011 | F1: 0.0055 | LR: 0.000161
   Epoch 35/40 | Train: 0.0007 | Val: 0.0011 | F1: 0.0058 | LR: 0.000042
   Epoch 40/40 | Train: 0.0007 | Val: 0.0011 | F1: 0.0059 | LR: 0.000000

✅ Training completed in 3607.5s (60.1 min)
   Best Val F1: 0.0059


In [ ]:
# Evaluate Model & Find Optimal Threshold
print('\n📈 Evaluating model...')

model.eval()

def predict_batch(X):
    """Predict probabilities with model"""
    X_tensor = torch.FloatTensor(X).to(device)
    with torch.no_grad():
        outputs = model(X_tensor)
        probs = torch.sigmoid(outputs).cpu().numpy()
    return probs

# Get predictions
val_probs = predict_batch(X_val)
train_probs = predict_batch(X_train)

#  Find Optimal Threshold - Finer search
print('\n🎯 Finding optimal threshold...')

from sklearn.metrics import f1_score, precision_score, recall_score

best_f1 = 0
best_threshold = 0.01
results = []

# Finer threshold search
for threshold in [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1, 0.12, 0.15]:
    val_preds = (val_probs >= threshold).astype(np.int8)
    
    if val_preds.sum() == 0:
        continue
    
    f1_micro = f1_score(y_val, val_preds, average='micro', zero_division=0)
    precision = precision_score(y_val, val_preds, average='micro', zero_division=0)
    recall = recall_score(y_val, val_preds, average='micro', zero_division=0)
    coverage = val_preds.sum(axis=1).mean()
    
    results.append({
        'threshold': threshold,
        'f1': f1_micro,
        'precision': precision,
        'recall': recall,
        'coverage': coverage
    })
    
    print(f'   Threshold {threshold:.3f}: F1={f1_micro:.4f}, P={precision:.4f}, R={recall:.4f}, Avg={coverage:.1f}')
    
    if f1_micro > best_f1:
        best_f1 = f1_micro
        best_threshold = threshold

# Use slightly lower threshold for submissio
# Kaggle metric often benefits from more predictions with good precision
PROB_THRESHOLD = max(0.03, best_threshold - 0.02)
print(f'\n✅ Best threshold: {best_threshold} (F1={best_f1:.4f})')
print(f'   Using threshold for submission: {PROB_THRESHOLD}')

# Final evaluation
val_preds = (val_probs >= PROB_THRESHOLD).astype(np.int8)
train_preds = (train_probs >= PROB_THRESHOLD).astype(np.int8)

val_hamming = hamming_loss(y_val, val_preds)
train_hamming = hamming_loss(y_train, train_preds)
final_f1 = f1_score(y_val, val_preds, average='micro', zero_division=0)

print(f'\n📊 Final Results (threshold={PROB_THRESHOLD}):')
print(f'   Train Hamming Loss: {train_hamming:.6f}')
print(f'   Val Hamming Loss: {val_hamming:.6f}')
print(f'   Val F1 (micro): {final_f1:.4f}')
print(f'   Val Predictions: {val_preds.sum():,} positives ({100*val_preds.mean():.2f}%)')
print(f'   Unique GO terms predicted: {(val_preds.sum(axis=0) > 0).sum():,}')


📈 Evaluating model...

🎯 Finding optimal threshold...
   Threshold 0.010: F1=0.0044, P=0.0022, R=1.0000, Avg=1984.0
   Threshold 0.020: F1=0.0046, P=0.0023, R=0.9996, Avg=1909.3
   Threshold 0.030: F1=0.0049, P=0.0025, R=0.9986, Avg=1782.5
   Threshold 0.040: F1=0.0053, P=0.0027, R=0.9962, Avg=1627.9
   Threshold 0.050: F1=0.0059, P=0.0030, R=0.9920, Avg=1464.6
   Threshold 0.060: F1=0.0066, P=0.0033, R=0.9862, Avg=1303.9
   Threshold 0.070: F1=0.0074, P=0.0037, R=0.9792, Avg=1152.6
   Threshold 0.080: F1=0.0084, P=0.0042, R=0.9715, Avg=1013.6
   Threshold 0.090: F1=0.0094, P=0.0047, R=0.9620, Avg=887.6
   Threshold 0.100: F1=0.0107, P=0.0054, R=0.9510, Avg=775.4
   Threshold 0.120: F1=0.0137, P=0.0069, R=0.9249, Avg=588.0
   Threshold 0.150: F1=0.0198, P=0.0100, R=0.8793, Avg=384.4

✅ Best threshold: 0.15 (F1=0.0198)
   Using threshold for submission: 0.13

📊 Final Results (threshold=0.13):
   Train Hamming Loss: 0.249635
   Val Hamming Loss: 0.253632
   Val F1 (micro): 0.0154
   Val

## 🎯 BƯỚC 6: Generate Test Predictions

In [13]:
print('='*60)
print('🎯 GENERATING TEST PREDICTIONS')
print('='*60)

# Extract features for test proteins with ENHANCED function
print('\n🧬 Extracting test features (enhanced)...')

test_protein_ids = list(test_sequences.keys())
X_test_features = []

for pid in tqdm(test_protein_ids, desc='Extracting test features'):
    X_test_features.append(extract_features_enhanced(test_sequences[pid]))

X_test = np.array(X_test_features)
X_test_scaled = scaler.transform(X_test)

print(f'✅ Test features: {X_test_scaled.shape}')

🎯 GENERATING TEST PREDICTIONS

🧬 Extracting test features (enhanced)...


Extracting test features: 100%|██████████| 224309/224309 [09:57<00:00, 375.53it/s]


✅ Test features: (224309, 150)


In [ ]:
# Make Predictions (Batched)
print('\n🚀 Making predictions...')

model.eval()
batch_size = 10000
all_probs = []

for i in tqdm(range(0, len(X_test_scaled), batch_size), desc='Predicting'):
    batch_X = X_test_scaled[i:i+batch_size]
    X_tensor = torch.FloatTensor(batch_X).to(device)
    
    with torch.no_grad():
        outputs = model(X_tensor)
        probs = torch.sigmoid(outputs).cpu().numpy()
    all_probs.append(probs)

y_test_proba = np.vstack(all_probs)
print(f'✅ Predictions shape: {y_test_proba.shape}')


🚀 Making predictions...


Predicting: 100%|██████████| 23/23 [00:43<00:00,  1.90s/it]


✅ Predictions shape: (224309, 2000)


## 📝 BƯỚC 7: Create Submission File

In [ ]:
# CREATE SUBMISSION WITH SMARTER GO PROPAGATION
print('='*60)
print('📝 CREATING SUBMISSION FILE (OPTIMIZED)')
print('='*60)

import os
import shutil

# XÓA FILE CŨ TRƯỚC KHI TẠO MỚI
submission_file = SUBMISSION_PATH / 'submission.tsv'
propagated_file = SUBMISSION_PATH / 'submission_propagated.tsv'

for f in [submission_file, propagated_file]:
    if f.exists():
        os.remove(f)
        print(f'   🗑️ Đã xóa file cũ: {f}')

# TERMS TO EXCLUDE OR DOWN-WEIGHT
# Root terms -
ROOT_TERMS = {
    'GO:0008150',  # biological_process
    'GO:0003674',  # molecular_function  
    'GO:0005575',  # cellular_component
}

# High-level terms to EXCLUDE
EXCLUDE_GENERAL_TERMS = {
    'GO:0110165',  # cellular anatomical structure
    'GO:0005488',  # binding
    'GO:0005623',  # cell
    'GO:0044464',  # cell part
    'GO:0044424',  # intracellular part
    'GO:0005622',  # intracellular
    'GO:0009987',  # cellular process
    'GO:0008152',  # metabolic process
}

# Terms to DOWN-WEIGHT significantly
DOWNWEIGHT_HEAVY = {
    'GO:0005515': 0.3,   # protein binding
    'GO:0005737': 0.4,   # cytoplasm
    'GO:0005634': 0.5,   # nucleus
    'GO:0016020': 0.5,   # membrane
    'GO:0005829': 0.4,   # cytosol
    'GO:0005886': 0.5,   # plasma membrane
    'GO:0043226': 0.5,   # organelle
    'GO:0043227': 0.5,   # membrane-bounded organelle
    'GO:0043229': 0.5,   # intracellular organelle
    'GO:0043231': 0.5,   # intracellular membrane-bounded organelle
}

# All terms to filter out
ALL_EXCLUDE = ROOT_TERMS | EXCLUDE_GENERAL_TERMS

#  GO HIERARCHY PROPAGATION - SMARTER VERSION
if GO_HIERARCHY_LOADED:
    print('\n🌳 Applying Smart GO Hierarchy Propagation...')
    
    # Build IA dictionary
    if ia_weights is not None:
        ia_dict = dict(zip(ia_weights['go_term'], ia_weights['ia_weight']))
        max_ia = max(ia_dict.values()) if ia_dict else 1.0
    else:
        ia_dict = {}
        max_ia = 1.0
    
    # Build filtered ancestors
    print('   Pre-computing mappings...')
    term_ancestors_filtered = {}
    for term in filtered_go_terms_final:
        ancestors = ancestor_cache.get(term, {term})
        # Filter out root and general terms
        filtered_ancestors = ancestors - ALL_EXCLUDE
        term_ancestors_filtered[term] = filtered_ancestors
    
    # Collect all possible terms
    all_terms_in_ancestors = set()
    for ancestors in term_ancestors_filtered.values():
        all_terms_in_ancestors.update(ancestors)
    all_terms_in_ancestors -= ALL_EXCLUDE
    
    print(f'   Model GO terms: {len(filtered_go_terms_final):,}')
    print(f'   Possible output terms: {len(all_terms_in_ancestors):,}')
    print(f'   Excluded general terms: {len(ALL_EXCLUDE)}')
    
    # Precompute term weights (for down-weighting)
    term_weights = {}
    for t in all_terms_in_ancestors:
        if t in DOWNWEIGHT_HEAVY:
            term_weights[t] = DOWNWEIGHT_HEAVY[t]
        else:
            term_weights[t] = 1.0
    
    # Precompute IA weights
    if ia_dict:
        term_ia_final = {t: ia_dict.get(t, 1.0) / max_ia for t in all_terms_in_ancestors}
    else:
        term_ia_final = None
    
    # DYNAMIC THRESHOLDING
    # Use per-protein adaptive thresholding
    PROPAGATION_THRESHOLD = 0.02  # Lower base threshold
    MAX_PREDS_PROPAGATED = 150    # More predictions
    MIN_PREDS_PER_PROTEIN = 10    # Ensure minimum predictions
    
    print(f'   Base threshold: {PROPAGATION_THRESHOLD}')
    print(f'   Max per protein: {MAX_PREDS_PROPAGATED}')
    print(f'   Min per protein: {MIN_PREDS_PER_PROTEIN}')
    print(f'   Processing {len(test_protein_ids):,} proteins...\n')
    
    total_predictions = 0
    go_term_counts = {}
    
    with open(propagated_file, 'w') as f:
        for i in tqdm(range(len(test_protein_ids)), desc='Creating submission', mininterval=0.5):
            protein_id = test_protein_ids[i]
            probs = y_test_proba[i]
            
            # Get top predictions with LOWER threshold
            top_k = 50  # Consider more candidates
            top_indices = np.argsort(-probs)[:top_k]
            
            # Adaptive threshold: use model's top prediction as reference
            max_prob = probs[top_indices[0]]
            adaptive_threshold = min(PROPAGATION_THRESHOLD, max_prob * 0.3)
            
            # Filter by threshold
            mask = probs[top_indices] >= adaptive_threshold
            if mask.sum() >= MIN_PREDS_PER_PROTEIN:
                selected_indices = top_indices[mask]
            else:
                # Keep at least MIN_PREDS_PER_PROTEIN
                selected_indices = top_indices[:MIN_PREDS_PER_PROTEIN]
            
            # Propagate to ancestors with filtering
            term_scores = {}
            for idx in selected_indices:
                term = filtered_go_terms_final[idx]
                prob = float(probs[idx])
                
                # Only propagate if term is in our filtered set
                if term in term_ancestors_filtered:
                    for ancestor in term_ancestors_filtered[term]:
                        if ancestor in ALL_EXCLUDE:
                            continue
                        
                        # Apply down-weight
                        weight = term_weights.get(ancestor, 1.0)
                        weighted_prob = prob * weight
                        
                        if ancestor not in term_scores or weighted_prob > term_scores[ancestor]:
                            term_scores[ancestor] = weighted_prob
            
            # Sort and limit predictions
            sorted_items = sorted(term_scores.items(), key=lambda x: -x[1])
            
            # Filter out very low confidence predictions
            min_conf = sorted_items[0][1] * 0.1 if sorted_items else 0
            sorted_items = [(t, p) for t, p in sorted_items if p >= min_conf]
            sorted_items = sorted_items[:MAX_PREDS_PROPAGATED]
            
            # Write predictions
            for term, prob in sorted_items:
                # Apply IA weight adjustment
                if term_ia_final and term in term_ia_final:
                    ia_w = term_ia_final[term]
                    # Higher IA = more informative = boost slightly
                    final_prob = prob * (0.8 + 0.4 * ia_w)
                else:
                    final_prob = prob
                
                # Clamp probability
                final_prob = min(1.0, max(0.001, final_prob))
                
                f.write(f'{protein_id}\t{term}\t{final_prob:.6f}\n')
                total_predictions += 1
                go_term_counts[term] = go_term_counts.get(term, 0) + 1
    
    print(f'\n📊 Results:')
    print(f'   Total predictions: {total_predictions:,}')
    print(f'   Unique GO terms: {len(go_term_counts):,}')
    print(f'   Avg per protein: {total_predictions / len(test_protein_ids):.1f}')
    
    # Show distribution of top terms
    print(f'\n🔍 Top 15 GO terms:')
    sorted_terms = sorted(go_term_counts.items(), key=lambda x: -x[1])[:15]
    for i, (term, count) in enumerate(sorted_terms, 1):
        name = go_names.get(term, '')[:30]
        pct = 100 * count / len(test_protein_ids)
        # Check if down-weighted
        weight = term_weights.get(term, 1.0)
        weight_str = f'(w={weight})' if weight < 1.0 else ''
        print(f'   {i:2d}. {term}: {count:,} ({pct:.1f}%) - {name} {weight_str}')
    
    # Check if too many general terms
    general_in_top = sum(1 for t, _ in sorted_terms[:10] if t in DOWNWEIGHT_HEAVY)
    if general_in_top > 5:
        print(f'\n   ⚠️ Warning: {general_in_top}/10 top terms are general (down-weighted)')
    
    # Copy to final submission
    shutil.copy(propagated_file, submission_file)
    
else:
    print('\n⚠️ GO hierarchy not loaded - creating simple submission...')
    
    SIMPLE_THRESHOLD = 0.02
    MAX_PREDS = 150
    total_predictions = 0
    
    with open(submission_file, 'w') as f:
        for i in tqdm(range(len(test_protein_ids)), desc='Creating submission'):
            protein_id = test_protein_ids[i]
            probs = y_test_proba[i]
            
            top_indices = np.argsort(-probs)[:MAX_PREDS]
            mask = probs[top_indices] >= SIMPLE_THRESHOLD
            if mask.sum() >= 10:
                top_indices = top_indices[mask]
            else:
                top_indices = top_indices[:10]
            
            for idx in top_indices:
                term = filtered_go_terms_final[idx]
                prob = float(probs[idx])
                f.write(f'{protein_id}\t{term}\t{prob:.6f}\n')
                total_predictions += 1
    
    print(f'   Total predictions: {total_predictions:,}')

# Verify file
file_size = os.path.getsize(submission_file) / (1024 * 1024)
print(f'\n✅ Submission saved: {submission_file}')
print(f'   Size: {file_size:.2f} MB')

📝 CREATING SUBMISSION FILE (OPTIMIZED)

🌳 Applying Smart GO Hierarchy Propagation...
   Pre-computing mappings...
   Model GO terms: 2,000
   Possible output terms: 3,906
   Excluded general terms: 11
   Base threshold: 0.02
   Max per protein: 150
   Min per protein: 10
   Processing 224,309 proteins...



Creating submission: 100%|██████████| 224309/224309 [03:09<00:00, 1180.65it/s]



📊 Results:
   Total predictions: 33,506,076
   Unique GO terms: 3,906
   Avg per protein: 149.4

🔍 Top 15 GO terms:
    1. GO:0050896: 201,593 (89.9%) - response to stimulus 
    2. GO:0065007: 190,209 (84.8%) - biological regulation 
    3. GO:0050789: 182,277 (81.3%) - regulation of biological proce 
    4. GO:0050794: 173,478 (77.3%) - regulation of cellular process 
    5. GO:0006950: 170,045 (75.8%) - response to stress 
    6. GO:0071944: 169,772 (75.7%) - cell periphery 
    7. GO:0051716: 168,177 (75.0%) - cellular response to stimulus 
    8. GO:0042802: 154,797 (69.0%) - identical protein binding 
    9. GO:0032502: 152,239 (67.9%) - developmental process 
   10. GO:0044238: 149,831 (66.8%) - primary metabolic process 
   11. GO:0032501: 140,234 (62.5%) - multicellular organismal proce 
   12. GO:0048856: 140,029 (62.4%) - anatomical structure developme 
   13. GO:0042221: 138,618 (61.8%) - response to chemical 
   14. GO:0071840: 135,138 (60.2%) - cellular component organiz

In [16]:
# Preview submission
print('\n📄 Submission Preview:')
preview_df = pd.read_csv(submission_file, sep='\t', header=None, names=['protein_id', 'go_term', 'confidence'], nrows=10)
print(preview_df.to_string(index=False))

print('\n🏆 Top GO Terms in submission:')
# Read just go_term column for stats
go_counts = pd.read_csv(submission_file, sep='\t', header=None, usecols=[1], names=['go_term'])['go_term'].value_counts().head(10)
for i, (term, count) in enumerate(go_counts.items(), 1):
    name = go_names.get(term, 'Unknown') if GO_HIERARCHY_LOADED else ''
    print(f'   {i}. {term}: {count:,} ({name[:40]}...)')


📄 Submission Preview:
protein_id    go_term  confidence
A0A0C5B5G6 GO:0003677    0.482135
A0A0C5B5G6 GO:0003676    0.510117
A0A0C5B5G6 GO:0065007    0.473632
A0A0C5B5G6 GO:0080090    0.462964
A0A0C5B5G6 GO:0050789    0.458164
A0A0C5B5G6 GO:0048522    0.458286
A0A0C5B5G6 GO:0051252    0.459022
A0A0C5B5G6 GO:0006357    0.468526
A0A0C5B5G6 GO:1902680    0.457236
A0A0C5B5G6 GO:0019222    0.471166

🏆 Top GO Terms in submission:
   1. GO:0050896: 201,593 (response to stimulus...)
   2. GO:0065007: 190,209 (biological regulation...)
   3. GO:0050789: 182,277 (regulation of biological process...)
   4. GO:0050794: 173,478 (regulation of cellular process...)
   5. GO:0006950: 170,045 (response to stress...)
   6. GO:0071944: 169,772 (cell periphery...)
   7. GO:0051716: 168,177 (cellular response to stimulus...)
   8. GO:0042802: 154,797 (identical protein binding...)
   9. GO:0032502: 152,239 (developmental process...)
   10. GO:0044238: 149,831 (primary metabolic process...)


## ✅ BƯỚC 8: Summary

In [17]:
print('\n' + '='*60)
print('🎉 PIPELINE COMPLETE!')
print('='*60)

# Get stats from file
import os
file_size = os.path.getsize(submission_file) / (1024 * 1024)  # MB

print(f'''
📊 SUMMARY:
{'='*50}

📁 Data:
   • Training proteins: {len(valid_proteins):,}
   • Test proteins: {len(test_protein_ids):,}
   • GO terms trained: {len(filtered_go_terms_final):,}

🧬 Features: {X_scaled.shape[1]} dimensions
   • AAC, Physicochemical, Dipeptides, PseAAC

🤖 Model: Neural Network
   • Hidden: {HIDDEN_DIM}, ResBlocks: 4
   • Focal Loss + IA Weights
   • Training time: {training_time/60:.1f} min

🌳 GO Propagation: {"✅ ENABLED" if GO_HIERARCHY_LOADED else "❌"}

📝 Submission:
   • File: {submission_file}
   • Size: {file_size:.1f} MB

{'='*50}
✅ Ready for Kaggle submission!
''')


🎉 PIPELINE COMPLETE!

📊 SUMMARY:

📁 Data:
   • Training proteins: 82,404
   • Test proteins: 224,309
   • GO terms trained: 2,000

🧬 Features: 150 dimensions
   • AAC, Physicochemical, Dipeptides, PseAAC

🤖 Model: Neural Network
   • Hidden: 768, ResBlocks: 4
   • Focal Loss + IA Weights
   • Training time: 60.1 min

🌳 GO Propagation: ✅ ENABLED

📝 Submission:
   • File: /kaggle/working/submission.tsv
   • Size: 865.3 MB

✅ Ready for Kaggle submission!

